In [6]:
%%capture
%pip install -q torch torchvision torchaudio
%pip install -q opencv-python numpy matplotlib pillow tqdm
%pip install -q scikit-image
%pip install -q dlib
%pip install -q face-alignment
%pip install -q opencv-python tqdm


In [9]:
import os
import cv2
from tqdm import tqdm
from multiprocessing import Pool

### Bước 1: Tiền xử lý
-	Sử dụng một bộ phát hiện khuôn mặt (dùng OpenCV frontal face cascade classifier như trong bài báo) để tìm vùng khuôn mặt trong mỗi ảnh.
-	Cắt vùng mặt đã phát hiện được.
-	Thay đổi kích thước (resize) tất cả các ảnh mặt đã cắt về kích thước 128 x 128. 
-   Lưu ảnh đã cắt ra một thư mục mới có tên "128_crop_dataset"

Chú thích: Mã nguồn sử dụng bộ đa luồng multiprocessing của python để tăng tốc



In [ ]:
input_folder = "CelabA_dataset\img_align_celeba\img_align_celeba"
output_folder = "128_crop_dataset"


In [11]:

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

profile_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "lbpcascade_profileface.xml"
)


In [12]:
def process_image(filename):
    img_path = os.path.join(input_folder, filename)
    img = cv2.imread(img_path)

    if img is None:
        return

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Phát hiện frontal faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    # Nếu không có frontal → thử profile face
    if len(faces) == 0:
        faces = profile_cascade.detectMultiScale(gray, 1.1, 4)

    if len(faces) == 0:
        return  # bỏ qua ảnh không tìm được mặt

    # chọn mặt lớn nhất
    x, y, w, h = sorted(faces, key=lambda b: b[2]*b[3], reverse=True)[0]

    # mở rộng khung crop
    pad_w = int(0.2 * w)
    pad_h = int(0.2 * h)

    x1 = max(0, x - pad_w)
    y1 = max(0, y - pad_h)
    x2 = min(img.shape[1], x + w + pad_w)
    y2 = min(img.shape[0], y + h + pad_h)

    face_crop = img[y1:y2, x1:x2]

    # resize về 128x128
    face_resized = cv2.resize(face_crop, (128, 128))

    # lưu ảnh
    output_path = os.path.join(output_folder, filename)
    cv2.imwrite(output_path, face_resized)


In [ ]:
if __name__ == "__main__":
    file_list = sorted([f for f in os.listdir(input_folder) if f.endswith(".jpg")])

    # số worker (4 hoặc 8 tùy CPU)
    num_workers = 8

    print(f"Processing {len(file_list)} images using {num_workers} workers...")

    with Pool(num_workers) as p:
        list(tqdm(p.imap(process_image, file_list), total=len(file_list)))

    print("Done!")
